In [1]:
import tkinter as tk

In [2]:
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from PIL import ImageTk,Image

In [12]:

####################################################################################################
# reading data ##

df=pd.read_csv('ml-100k/u.data',sep='\t',names=['user_id','item_id','rating','ts'])

cols='item_id|title|release date|video release date|\
              IMDb URL|unknown|Action|Adventure|Animation|\
              Childrens|Comedy|Crime|Documentary|Drama|Fantasy|\
              Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|\
              Thriller|War|Western|'.split('|')
movies=pd.read_csv('ml-100k/u.item',sep='|',names=cols)

data=pd.merge(df.drop('ts',axis=1),movies[['item_id','title']],on='item_id')

rate_count=data.groupby('title')['rating'].count()

df_pivot = data.pivot_table(index='user_id',columns='title',values='rating')
#######################################################################################################
# setting window ##

app=tk.Tk()
app.geometry('400x300')
app.title('Movie recommendation system')
########################################################################################################
# setting image canvas ##

bg = ImageTk.PhotoImage(Image.open('mv.jpg'))
canvas1 = tk.Canvas( app, width = 400,height = 400)
canvas1.pack(fill = "both", expand = True)
canvas1.create_image( 0, 0, image = bg, anchor = "nw")
canvas1.create_text( 300, 250, text = "Have a nice day !!",fill="white",font="Times 10 italic bold")
b="Press the below button  to \nsee movies we recommend \nto you"
canvas1.create_text( 320, 60, text = b ,fill="white",font="Calibri 10  italic bold")
canvas1.create_text( 60, 15, text ="Movies List" ,fill="white",font="Calibri 12   bold")

###########################################################################################################
############################################################################################################
# setting correlation concept for recommendation sysytem ###

listbox = tk.Listbox(app,height=15,width=30,bg='black', fg='white')
listbox.place(x=20,y=30)
for i,t in enumerate (data['title'].unique()):
    listbox.insert(i+1, t)

def recommend():
    inp = listbox.get(listbox.curselection())
    sim_df=pd.DataFrame(df_pivot.corrwith(df_pivot[inp]),columns=['Correlation'])
    sim_df['count'] = rate_count
    recom=sim_df[(sim_df['count']>100)].sort_values(by='Correlation',ascending=False).head(10)
    if inp == recom.index[0]:
        recom.drop(inp,axis=0,inplace=True)
    recommended=recom.index[:3]  
    result1.set(recommended[0])                   
    result2.set(recommended[1])  
    result3.set(recommended[2])                      
    print(inp)
    print(recommended)
tk.Button(app,text='Show recommendations',command=recommend,bg='red',fg='white').place(x=250,y=100) 

result1= tk.Variable(app)
result1.set('Movie 1')
tk.Label(app,textvariable=result1,fg="red").place(x=250,y=140)
    
result2= tk.Variable(app)
result2.set('Movie 2')
tk.Label(app,textvariable=result2,fg="red").place(x=250,y=170)


result3= tk.Variable(app)
result3.set('Movie 3')
tk.Label(app,textvariable=result3,fg="red").place(x=250,y=200)

app.mainloop()